In [1]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_matching_tools.io.graph_dataset import GraphDataset
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
from graph_matching_tools.algorithms.multiway.hippi import hippi_multiway_matching
from graph_matching_tools.algorithms.kernels.gaussian import create_gaussian_node_kernel
from graph_matching_tools.algorithms.kernels.utils import create_full_node_affinity_matrix
from graph_matching_tools.algorithms.multiway.stiefel import sparse_stiefel_manifold_sync

import scipy.io as sio
# from KerGM_python.grackel import *
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/KerGM_python/grackel/")
import re
import KerGM_python.grackel.grackelpy.algorithms.matching as match
import KerGM_python.grackel.grackelpy.algorithms.multimatch as mm
import KerGM_python.grackel.grackelpy.algorithms.quickmatch as qm
import KerGM_python.grackel.grackelpy.utils.measures as measures
import KerGM_python.grackel.grackelpy.utils.kernels as kern
import KerGM_python.grackel.grackelpy.utils.utils as utils
import KerGM_python.grackel.graph_processing.graph_processing as gp
from os.path import exists

In [2]:
def get_permutation_matrix_from_dictionary(matching, g_sizes):
    """
    Create the full permutation matrix from the matching result
    :param matching: the matching result for each graph (nodes number, assignment)
    :param g_sizes: the list of the size of the different graph
    :return: the full permutation matrix
    """
    f_size = int(np.sum(g_sizes))
    res = np.zeros((f_size, f_size))

    idx1 = 0
    for i_g1 in range(len(g_sizes)):
        idx2 = 0
        for i_g2 in range(len(g_sizes)):
            match = matching["{},{}".format(i_g1, i_g2)]
            for k in match:
                res[idx1 + int(k), idx2 + match[k]] = 1
            idx2 += g_sizes[i_g2]
        idx1 += g_sizes[i_g1]
        
    np.fill_diagonal(res,1)
    return res

In [3]:
def create_full_adjacency_matrix(graphs):
    """
    Create the full adjacency matrix with the matrices on the diagonal

    :param list graphs: the list of graphs
    :return:
    """
    sizes = []
    full_size = 0
    for g in graphs:
        size = nx.number_of_nodes(g)
        sizes.append(size)
        full_size += size

    a = np.zeros((full_size, full_size))

    index = 0
    for i in range(len(graphs)):
        adj = (nx.adjacency_matrix(graphs[i])).toarray()
        a[index:index+sizes[i], index:index+sizes[i]] = adj
        index += sizes[i]

    return a, sizes

In [4]:
path_to_graph_folder = '/home/rohit/PhD_Work/GM_my_version/Graph_matching/data/simu_graph/simu_test_single_noise/'
path_to_dummy_graphs_folder = '/home/rohit/PhD_Work/GM_my_version/Graph_matching/data/simu_graph/test_with_dummy/'

In [5]:
trials = np.sort(os.listdir(path_to_graph_folder))

sigma = 200
u_dim = 100 # dim for universe of nodes

scores = {100:[],400:[],700:[],1000:[],1300:[]}
prec_scores = {100:[],400:[],700:[],1000:[],1300:[]}
rec_scores = {100:[],400:[],700:[],1000:[],1300:[]}

for trial in trials:
    
    if float(trial) <= 0.3:
        
        print('trial: ', trial)

        all_files = os.listdir(path_to_graph_folder+trial)

        for folder in all_files:

            if os.path.isdir(path_to_graph_folder+trial+'/'+ folder):

                print('Noise folder: ',folder)
                
#                 path_to_dummy_graphs = path_to_dummy_graphs_folder + '/' + trial + '/' + folder+'/0/graphs/'
                path_to_graphs = path_to_graph_folder + '/' + trial + '/' + folder+'/graphs/'
                path_to_groundtruth_ref = path_to_graph_folder + '/' + trial +'/' + folder + '/permutation_to_ref_graph.gpickle'
                path_to_groundtruth  = path_to_graph_folder + '/' + trial + '/' + folder + '/ground_truth.gpickle'
            
        
                noise = folder.split(',')[0].split('_')[1]
                
                if exists(path_to_graph_folder + '/' + trial + '/' + folder +'/X_HiPPi.mat'):   
                    continue
            
            
                graph_meta = GraphDataset(path_to_graphs, path_to_groundtruth_ref)
                ground_truth =  nx.read_gpickle(path_to_groundtruth)
    
#                 graphs = [nx.read_gpickle(path_to_dummy_graphs + graph) for graph in np.sort(os.listdir(path_to_dummy_graphs))]
#                 sizes = [nx.number_of_nodes(graph) for graph in graphs]
        
                res = get_permutation_matrix_from_dictionary(ground_truth, graph_meta.sizes)
                
                node_kernel = create_gaussian_node_kernel(sigma,'coord')
                knode = create_full_node_affinity_matrix(graph_meta.list_graphs, node_kernel)
                
                s = np.eye(knode.shape[0], knode.shape[1])
#                 path_init = path_to_graph_folder + '/'+ trial + '/' + folder + '/X_kmeans.mat'                
#                 X_kmeans = sio.loadmat(path_init)
                #u = X_kmeans['U']
#                 x = X_kmeans['full_assignment_mat']
                

                # Old Implementation
#                 node_kernel = kern.create_node_kernel(sigma, "coord")   #Change coord to sphere_3dcoords for real data
#                 knode = mm.create_full_node_affinity_matrix(graph_meta.list_graphs, node_kernel)
#                 s, _ = utils.create_full_adjacency_matrix(graph_meta.list_graphs)
#                 s = np.eye(knode.shape[0], knode.shape[1])
#                 u = mm.hippi_multiway_matching(s, graph_meta.sizes, knode, u_dim)
#                 P = u @ u.T

#                 rng = np.random.default_rng()
#                 u = rng.uniform(size=(s.shape[0], u_dim))


                #u = sparse_stiefel_manifold_sync(x, u_dim, graph_meta.sizes)
                #print('u shape', u.shape)
        
                U = hippi_multiway_matching(s, graph_meta.sizes, knode, u_dim)

                P = U @ U.T

                HiPPi_X = {}
                HiPPi_X['full_assignment_mat'] = P
                
                print('P shape', P.shape)
                print('s shape', s.shape)

                sio.savemat(path_to_graph_folder + '/' + trial + '/' + folder + '/X_HiPPi.mat',HiPPi_X)

                f1, prec, rec = matching.compute_f1score(P,res)

                scores[int(noise)].append(f1)
                prec_scores[int(noise)].append(prec)
                rec_scores[int(noise)].append(rec)

trial:  0.0
Noise folder:  noise_100,outliers_varied
Noise folder:  noise_400,outliers_varied
Noise folder:  noise_700,outliers_varied
Noise folder:  noise_1300,outliers_varied
Noise folder:  noise_1000,outliers_varied
trial:  0.1
Noise folder:  noise_100,outliers_varied
Noise folder:  noise_400,outliers_varied
Noise folder:  noise_700,outliers_varied
Noise folder:  noise_1300,outliers_varied
Noise folder:  noise_1000,outliers_varied
trial:  0.2
Noise folder:  noise_100,outliers_varied
P shape (11427, 11427)
s shape (11427, 11427)
Noise folder:  noise_400,outliers_varied
P shape (11323, 11323)
s shape (11323, 11323)
Noise folder:  noise_700,outliers_varied
P shape (11333, 11333)
s shape (11333, 11333)
Noise folder:  noise_1300,outliers_varied
P shape (11292, 11292)
s shape (11292, 11292)
Noise folder:  noise_1000,outliers_varied
P shape (11262, 11262)
s shape (11262, 11262)
trial:  0.3
Noise folder:  noise_100,outliers_varied
P shape (11350, 11350)
s shape (11350, 11350)
Noise folder: 